### 1. invoke() vs stream() 방식 비교

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError(".env 파일에 OPENAI_API_KEY가 설정되지 않았습니다.")

print("✅ API 키가 성공적으로 로드되었습니다.")

✅ API 키가 성공적으로 로드되었습니다.


In [2]:
model = ChatOpenAI(model="gpt-4o-mini", streaming=True)
prompt = [HumanMessage(content="서울에서 두쫀쿠를 구하려고 하는데 어디에서 구할 수 있을까?")]

In [6]:
# 1. invoke 방식
reply = model.invoke(prompt)
print(reply.content)

'''
동작방식 : 모델이 응답을 완전히 생성한 후, 한번에 전체 결과를 반환
적합한 상황: 응답 시간이 덜 중요하고, 전체 응답을 한 번에 처리하는 것이 더 효율적인 경우
'''

서울에서 두쫀쿠를 구할 수 있는 곳은 여러 곳이 있습니다. 일반적으로 한국의 전통 시장이나 아시아 식료품점을 방문하면 두쫀쿠를 찾을 수 있습니다. 특히 다음과 같은 장소를 고려해 볼 수 있습니다:

1. **재래시장**: 남대문시장, 동대문시장, 광장시장 등에서 두쫀쿠를 판매하는 가게가 있을 수 있습니다.
2. **대형 마트**: 이마트, 홈플러스, 롯데마트와 같은 대형 마트의 아시아 식품 코너에서 구매할 수 있습니다.
3. **온라인 쇼핑몰**: 쿠팡, G마켓, 11번가와 같은 온라인 쇼핑몰에서도 두쫀쿠를 검색하여 주문할 수 있습니다.
4. **아시아 식료품점**: 서울의 다양한 아시아 식료품 전문점에서 두쫀쿠를 찾아볼 수 있습니다.

구체적인 매장에 따라 재고가 다를 수 있으니 전화로 확인해 보시는 것도 좋은 방법입니다.


'\n동작방식 : 모델이 응답을 완전히 생성한 후, 한번에 전체 결과를 반환\n적합한 상황: 응답 시간이 덜 중요하고, 전체 응답을 한 번에 처리하는 것이 더 효율적인 경우\n'

In [7]:
# 2. stream 방식
for chunk in model.stream(prompt):
    print(chunk.content, end="")
'''
동작방식: 모델이 토큰을 생성하는 즉시 실시간으로 전송
적합한 상황: 실시간 응답이 중요한 경우, 예를 들어 채팅 애플리케이션에서 사용자 경험을 향상시키고자 할 때
'''

서울에서 두쫀쿠를 구하려면 여러 곳에서 찾아볼 수 있습니다. 주로 아시아 마트나 한국의 전통 시장에서 구할 수 있는데, 다음과 같은 장소를 추천드립니다:

1. **대형 마트**: 이마트, 롯데마트, 홈플러스 등의 대형 마트에서는 두쫀쿠를 판매하는 경우가 많습니다.

2. **전통 시장**: 광장시장, 노량진수산시장, 동대문시장 등에서 두쫀쿠를 찾을 수 있습니다. 특히 전통 시장에서는 신선한 재료를 구할 수 있는 장점이 있습니다.

3. **온라인 쇼핑몰**: 쿠팡, G마켓, 11번가 등의 온라인 쇼핑몰에서도 두쫀쿠를 구매할 수 있습니다.

4. **아시아 식료품점**: 서울 내 아시아 식료품점에서도 두쫀쿠를 판매하는 곳이 많으니, 근처의 아시아 마트를 찾아보세요.

구매하기 전에 전화로 재고를 확인하거나 웹사이트에서 미리 검색해보는 것도 좋습니다.

'\n동작방식: 모델이 토큰을 생성하는 즉시 실시간으로 전송\n적합한 상황: 실시간 응답이 중요한 경우, 예를 들어 채팅 애플리케이션에서 사용자 경험을 향상시키고자 할 때\n'

### stream 모드로 보는 여행 챗봇 대화 흐름

In [8]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# 1. 도시별 관광명소 도구
@tool
def get_attractions(city: str) -> list[str]:
    """
    주어진 도시의 인기 관광명소 5곳을 반환합니다.
    """
    sample = {
        "파리": ["에펠탑", "루브르박물관", "노트르담 대성당", "몽마르트 언덕", "개선문"],
        "도쿄": ["도쿄타워", "센소지", "시부야 스크램블", "신주쿠 교엔", "아키하바라"],
        "뉴욕": ["타임스퀘어", "자유의 여신상", "센트럴파크", "엠파이어 스테이트 빌딩", "브로드웨이"],
    }
    return sample.get(city, ["정보가 없습니다"])

# 2. 항공권 가격 도구
@tool
def get_flight_price(depart: str, arrive: str, date: str) -> str:
    """
    출발지→도착지, 날짜별 예상 항공권 가격을 반환합니다.
    """
    # 주요 도시 간 기본 가격 (원화)
    prices = {
        ("서울", "도쿄"): 350000,
        ("서울", "파리"): 1200000,
        ("서울", "뉴욕"): 1500000,
        ("도쿄", "파리"): 1100000,
        ("뉴욕", "파리"): 800000,
    }
    
    # 경로 확인
    route = (depart, arrive)
    if route in prices:
        price = prices[route]
        return f"{depart}에서 {arrive}까지 {date} 항공편 예상 가격은 {format(price, ',')}원입니다."
    else:
        return f"죄송합니다. {depart}에서 {arrive}로 가는 항공편 정보가 없습니다."


# 3. 여행 일정 추천 도구
@tool
def make_itinerary(city: str, days: int) -> str:
    """city에서 days일간 추천 일정을 반환합니다."""
    # 주요 도시별 일정 아이디어
    city_activities = {
        "파리": [
            "에펠탑 방문", "루브르 박물관 관람", "노트르담 대성당 구경", 
            "몽마르트 언덕 산책", "개선문 방문", "세느강 크루즈", 
            "샹젤리제 쇼핑", "베르사유 궁전 투어", "오르세 미술관"
        ],
        "도쿄": [
            "메이지 신궁 방문", "시부야 스크램블 구경", "센소지 사원", 
            "도쿄 타워", "하라주쿠 쇼핑", "긴자 탐방", 
            "아키하바라 전자상가", "스미다강 크루즈", "신주쿠 교엔 공원"
        ],
        "뉴욕": [
            "타임스퀘어 구경", "센트럴 파크 산책", "자유의 여신상 방문", 
            "엠파이어 스테이트 빌딩", "브로드웨이 쇼 관람", "메트로폴리탄 미술관",
            "소호 지역 쇼핑", "브루클린 브릿지 산책", "첼시 마켓 방문"
        ]
    }
    
    # 도시 정보가 있는지 확인
    if city not in city_activities:
        return "해당 도시의 일정 정보가 없습니다."
    
    # 해당 도시의 활동 목록
    activities = city_activities[city]
    
    # 포맷팅된 문자열로 일정 생성
    result = f"## {city} {days}일 추천 여행 일정\n\n"
    
    for day in range(1, days + 1):
        # 아침/점심/저녁 활동 선택 (순환하면서)
        morning = activities[(day * 3 - 3) % len(activities)]
        afternoon = activities[(day * 3 - 2) % len(activities)]
        evening = activities[(day * 3 - 1) % len(activities)]
        
        result += f"### Day {day}\n"
        result += f"- 아침: {morning}\n"
        result += f"- 점심: {city} 현지 레스토랑에서 점심\n"
        result += f"- 오후: {afternoon}\n"
        result += f"- 저녁: {evening} 후 저녁 식사\n\n"
    
    return result

# 4. 최종적으로 사용할 도구 목록
tools = [get_attractions, get_flight_price, make_itinerary]

In [9]:
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

sys_msg = SystemMessage(content="당신은 여행 계획을 도와주는 여행 전문 에이전트입니다. 사용자의 요청을 이해하고 적절한 도구를 순차적으로 사용하여 여행 계획을 세워주세요.")

def assistant(state: MessagesState):
    messages = [sys_msg] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

tool_node = ToolNode(tools)

builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", tool_node)

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")

In [10]:
memory = MemorySaver()
react_graph_memory = builder.compile(checkpointer=memory)

messages = [HumanMessage(content=
    "첫째, 일본의 상위 5개 관광명소를 알려줘. "
    "그다음, 서울→도쿄 2025-06-01 항공권 예상 가격을 알려주고, "
    "마지막으로 일본 3일 일정 추천해줘."
)]

config = {"configurable": {"thread_id": "2"}}

for chunk in react_graph_memory.stream({"messages": messages}, config):
    for node_name, update in chunk.items(): 
        msgs = update["messages"]
        for m in msgs:
            m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_attractions (call_EzBZRY6QbeW6HOb7WsiTXcyI)
 Call ID: call_EzBZRY6QbeW6HOb7WsiTXcyI
  Args:
    city: 도쿄
================================= Tool Message =================================
Name: get_attractions

["도쿄타워", "센소지", "시부야 스크램블", "신주쿠 교엔", "아키하바라"]
================================== Ai Message ==================================
Tool Calls:
  get_flight_price (call_WtZUlQPhyNjSSH8jsH2qecsS)
 Call ID: call_WtZUlQPhyNjSSH8jsH2qecsS
  Args:
    depart: 서울
    arrive: 도쿄
    date: 2025-06-01
================================= Tool Message =================================
Name: get_flight_price

서울에서 도쿄까지 2025-06-01 항공편 예상 가격은 350,000원입니다.
================================== Ai Message ==================================
Tool Calls:
  make_itinerary (call_5PRFvtkqnZs6eeo9aNQdTyKe)
 Call ID: call_5PRFvtkqnZs6eeo9aNQdTyKe
  Args:
    city: 도쿄
    days: 3
================================= Too